In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from datetime import date
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm
from matplotlib import colors

### Create regression data

In [31]:
tfidf_path = '../../data/tfidf_toprev_conlen_full_v2/'

In [32]:
lda_path = '../../data/lda_jsd_toprev_conlen/'

In [33]:
tag_path = '../../data/tag_novelty_full/'

In [227]:
fandom_list = ['sherlock_holmes_&_related_fandoms']
#  'arthurian_mythology_&_related_fandoms',
#  'supernatural',
#  'haikyuu',
#  'kuroko_no_basuke',
#  'hamilton_miranda',
#  'dragon_age_all_media_types',
#  'the_walking_dead_&_related_fandoms',
#  'buffy_the_vampire_slayer',
#  'les_miserables_all_media_types',
#  'naruto',
#  'tolkien_j_r_r_works_&_related_fandoms',
#  'shakespare_william_works',
#  'hetalia_axis_powers',
#  'attack_on_titan',
#  'ms_paint_adventures',
#  'homestuck',
#  'marvel',
#  'bishoujo_senshi_sailor_moon',
#  'one_direction',
#  'sherlock_holmes_&_related_fandoms']

In [228]:
def first_rel(field):
    try:
        return field.split(',')[0]
    except:
        return field

In [229]:
def find_major_rel(df):
    df['Relationship'] = df.apply(lambda row: first_rel(row['Relationship']), axis=1)
    counts = df['Relationship'].value_counts()
    return counts.keys()[0:5]


In [230]:
df_all = []
for i,fandom in enumerate(fandom_list):
    df_tfidf = pd.read_csv(os.path.join(tfidf_path,fandom + '_temporal_tfidf_cos_toprev_conlen.tsv'), sep = '\t')
    del df_tfidf['index']
    df_lda = pd.read_csv(os.path.join(lda_path,fandom + '_temporal_lda_jsd_toprev_full.tsv'), sep = '\t')
    del df_lda['index']
    df_tag = pd.read_csv(os.path.join(tag_path,fandom + '_tag_novelty_conlen_full.tsv'), sep = '\t')
    df = pd.merge(df_tfidf, df_lda, on=['AdditionalTags', 'ArchiveWarnings', 'Author', 'Bookmarks', 'Category',\
 'ChapterIndex', 'Chapters' ,'Characters','Comments' ,'CompleteDate',\
 'Fandoms', 'Hits' ,'Kudos', 'Language', 'Notes' ,'PublishDate' ,'Rating',\
 'Relationship' ,'Summary' ,'Title' ,'URL' ,'UpdateDate' ,'Words'], how='inner')
    
    df['fandom_category'] = fandom


    
    freq_rel = find_major_rel(df)
    df['Relationship'] = df['Relationship'].apply(lambda x: first_rel(x))
    df['Freq_relationship'] = df['Relationship'].apply(lambda x: 1 if x in freq_rel else 0)
    del df['Relationship']
    df_all.append(df)
    

In [231]:
df_all = pd.concat(df_all)

In [232]:
len(df_all)

73741

In [233]:
del df_all['Language']
del df_all['Notes']
del df_all['Summary']
del df_all['Title']
# del df_all['URL']
del df_all['AdditionalTags']
del df_all['Fandoms']
del df_all['Characters']
del df_all['Words']

In [234]:
df_all['Category'].fillna('Unknown', inplace=True)
df_all['Category_F_F'] = df_all['Category'].apply(lambda x: 1 if 'F/F' in x else 0)
df_all['Category_F_M'] = df_all['Category'].apply(lambda x: 1 if 'F/M' in x else 0)
df_all['Category_Gen'] = df_all['Category'].apply(lambda x: 1 if 'Gen' in x else 0)
df_all['Category_M_M'] = df_all['Category'].apply(lambda x: 1 if 'M/M' in x else 0)
df_all['Category_Multi'] = df_all['Category'].apply(lambda x: 1 if 'Multi' in x else 0)
df_all['Category_Other'] = df_all['Category'].apply(lambda x: 1 if 'Other' in x else 0)
df_all['Category_Unknown'] = df_all['Category'].apply(lambda x: 1 if 'Unknown' in x else 0)
del df_all['Category']

In [235]:
df_all['ArchiveWarnings_underage'] = df_all['ArchiveWarnings'].apply(lambda x: 1 if 'Underage' in x else 0)
df_all['ArchiveWarnings_death'] = df_all['ArchiveWarnings'].apply(lambda x: 1 if 'Major Character Death' in x else 0)
df_all['ArchiveWarnings_choose_no'] = df_all['ArchiveWarnings'].apply(lambda x: 1 if 'Creator Chose Not To Use Archive Warnings' in x else 0)
df_all['ArchiveWarnings_no_apply'] = df_all['ArchiveWarnings'].apply(lambda x: 1 if 'No Archive Warnings Apply' in x else 0)
df_all['ArchiveWarnings_violence'] = df_all['ArchiveWarnings'].apply(lambda x: 1 if 'Graphic Depictions Of Violence' in x else 0)
df_all['ArchiveWarnings_noncon'] = df_all['ArchiveWarnings'].apply(lambda x: 1 if 'Rape/Non-Con' in x else 0)

del df_all['ArchiveWarnings']

In [236]:
df_all['ChapterIndex'].fillna(0, inplace=True)

In [237]:
df_all['Author'].fillna('Unknown', inplace=True)
counts = df_all['Author'].value_counts()
idx = counts[counts >= 10].index
df_all['author_fic_cnt'] = df_all['Author'].apply(lambda x: counts[x])
del df_all['Author']

In [238]:
df_all['Rating_E'] = df_all['Rating'].apply(lambda x: 1 if x == 'Explicit' else 0)
df_all['Rating_G'] = df_all['Rating'].apply(lambda x: 1 if x == 'General Audiences' else 0)
df_all['Rating_M'] = df_all['Rating'].apply(lambda x: 1 if x == 'Mature' else 0)
df_all['Rating_N'] = df_all['Rating'].apply(lambda x: 1 if x == 'Not Rated' else 0)
df_all['Rating_T'] = df_all['Rating'].apply(lambda x: 1 if x == 'Teen And Up Audiences' else 0)
del df_all['Rating']

In [239]:
df_all['Fandom_harry_potter'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'harry_potter' else 0)
df_all['Fandom_dcu'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'doctor_who_&_related_fandoms' else 0)
df_all['Fandom_doctor_who'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'doctor_who_&_related_fandoms' else 0)
df_all['Fandom_star_wars'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'star_wars_all_media_types' else 0)
df_all['Fandom_arthurian'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'arthurian_mythology_&_related_fandoms' else 0)
df_all['Fandom_supernatural'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'supernatural' else 0)
df_all['Fandom_haikyuu'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'haikyuu' else 0)
df_all['Fandom_kuroko_no_basuke'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'kuroko_no_basuke' else 0)
df_all['Fandom_hamilton_miranda'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'hamilton_miranda' else 0)
df_all['Fandom_dragon_age'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'dragon_age_all_media_types' else 0)
df_all['Fandom_the_walking_dead'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'the_walking_dead_&_related_fandoms' else 0)
df_all['Fandom_buffy'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'buffy_the_vampire_slayer' else 0)
df_all['Fandom_les_miserables'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'les_miserables_all_media_types' else 0)
df_all['Fandom_naruto'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'naruto' else 0)
df_all['Fandom_tolkien'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'tolkien_j_r_r_works_&_related_fandoms' else 0)
df_all['Fandom_shakespare'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'shakespare_william_works' else 0)
df_all['Fandom_hetalia'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'hetalia_axis_powers' else 0)
df_all['Fandom_attack_on_titan'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'attack_on_titan' else 0)
df_all['Fandom_ms_paint_adventures'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'ms_paint_adventures' else 0)
df_all['Fandom_homestuck'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'homestuck' else 0)
df_all['Fandom_marvel'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'marvel' else 0)
df_all['Fandom_sailor_moon'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'bishoujo_senshi_sailor_moon' else 0)
df_all['Fandom_one_direction'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'one_direction' else 0)
df_all['Fandom_sherlock'] = df_all['fandom_category'].apply(lambda x: 1 if x == 'sherlock_holmes_&_related_fandoms' else 0)

del df_all['fandom_category']

In [240]:
def date_today(cell):
    try:
        y, m, d = cell.split('-')
        return abs(date.today() - date(int(y), int(m), int(d))).days
    except:
        return float('nan')

In [241]:
df_all['PublishDate'] = df_all.apply(lambda row: date_today(row['PublishDate']), axis = 1)
df_all['CompleteDate'] = df_all.apply(lambda row: date_today(row['CompleteDate']), axis = 1)
df_all['UpdateDate'] = df_all.apply(lambda row: date_today(row['UpdateDate']), axis = 1)

In [242]:
def find_history(field_list):
    field_list = [10000000 if np.isnan(x) else x for x in field_list]
    return np.amin(field_list)

In [243]:
df_all['history'] = df_all.apply(lambda row: find_history([row['PublishDate'], row['CompleteDate'], row['UpdateDate']]), axis=1)

In [244]:
del df_all['PublishDate']
del df_all['CompleteDate']
del df_all['UpdateDate']

In [245]:
df_all = df_all.replace([np.inf, -np.inf], np.nan)

In [246]:
df_all = df_all.dropna()

In [247]:
len(df_all)

72218

In [248]:
df_all = df_all.rename(columns = {'history':'History', 'Cos': 'Term_novelty', 'JSD': 'Topic_novelty'})

In [249]:
df_all.sort_values(by = ['Kudos','Term_novelty'])[-10:]

,Bookmarks,ChapterIndex,Chapters,Comments,Hits,Kudos,URL,Term_novelty,Topic_novelty,Freq_relationship,...,Fandom_shakespare,Fandom_hetalia,Fandom_attack_on_titan,Fandom_ms_paint_adventures,Fandom_homestuck,Fandom_marvel,Fandom_sailor_moon,Fandom_one_direction,Fandom_sherlock,History
3545,210,3.0,13,22,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.237045,0.605213,1,...,0,0,0,0,0,0,0,0,1,2363.0
3549,210,4.0,13,21,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.305969,0.563810,1,...,0,0,0,0,0,0,0,0,1,2363.0
3554,210,5.0,13,16,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.380022,0.560017,1,...,0,0,0,0,0,0,0,0,1,2363.0
3568,210,9.0,13,20,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.636159,0.579782,1,...,0,0,0,0,0,0,0,0,1,2363.0
3565,210,8.0,13,15,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.763681,0.632382,1,...,0,0,0,0,0,0,0,0,1,2363.0
3542,210,2.0,13,13,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.777861,0.610712,1,...,0,0,0,0,0,0,0,0,1,2363.0
3535,210,1.0,13,24,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.868642,0.668992,1,...,0,0,0,0,0,0,0,0,1,2363.0
920,216,8.0,25,23,363073.0,8778.0,http://archiveofourown.org/works/173274/chapte...,0.263675,0.494076,1,...,0,0,0,0,0,0,0,0,1,2722.0
17811,206,3.0,57,62,451758.0,13322.0,http://archiveofourown.org/works/729134/chapte...,0.167777,0.649099,1,...,0,0,0,0,0,0,0,0,1,1583.0
42119,206,57.0,57,20,451758.0,13322.0,http://archiveofourown.org/works/729134/chapte...,0.232635,0.637923,1,...,0,0,0,0,0,0,0,0,1,1583.0


In [250]:
df_all.sort_values(by = ['Kudos','Topic_novelty'])[-10:]

,Bookmarks,ChapterIndex,Chapters,Comments,Hits,Kudos,URL,Term_novelty,Topic_novelty,Freq_relationship,...,Fandom_shakespare,Fandom_hetalia,Fandom_attack_on_titan,Fandom_ms_paint_adventures,Fandom_homestuck,Fandom_marvel,Fandom_sailor_moon,Fandom_one_direction,Fandom_sherlock,History
3549,210,4.0,13,21,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.305969,0.563810,1,...,0,0,0,0,0,0,0,0,1,2363.0
3568,210,9.0,13,20,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.636159,0.579782,1,...,0,0,0,0,0,0,0,0,1,2363.0
3545,210,3.0,13,22,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.237045,0.605213,1,...,0,0,0,0,0,0,0,0,1,2363.0
3542,210,2.0,13,13,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.777861,0.610712,1,...,0,0,0,0,0,0,0,0,1,2363.0
3565,210,8.0,13,15,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.763681,0.632382,1,...,0,0,0,0,0,0,0,0,1,2363.0
3535,210,1.0,13,24,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.868642,0.668992,1,...,0,0,0,0,0,0,0,0,1,2363.0
3562,210,7.0,13,13,162275.0,7713.0,http://archiveofourown.org/works/342490/chapte...,0.217140,0.673330,1,...,0,0,0,0,0,0,0,0,1,2363.0
920,216,8.0,25,23,363073.0,8778.0,http://archiveofourown.org/works/173274/chapte...,0.263675,0.494076,1,...,0,0,0,0,0,0,0,0,1,2722.0
42119,206,57.0,57,20,451758.0,13322.0,http://archiveofourown.org/works/729134/chapte...,0.232635,0.637923,1,...,0,0,0,0,0,0,0,0,1,1583.0
17811,206,3.0,57,62,451758.0,13322.0,http://archiveofourown.org/works/729134/chapte...,0.167777,0.649099,1,...,0,0,0,0,0,0,0,0,1,1583.0


In [251]:
for item in df_all.sort_values(by = ['Kudos','Term_novelty'])[-10:]['URL']:
    print(item)

http://archiveofourown.org/works/342490/chapters/557836
http://archiveofourown.org/works/342490/chapters/558615
http://archiveofourown.org/works/342490/chapters/559296
http://archiveofourown.org/works/342490/chapters/562480
http://archiveofourown.org/works/342490/chapters/561433
http://archiveofourown.org/works/342490/chapters/556681
http://archiveofourown.org/works/342490/chapters/555348
http://archiveofourown.org/works/173274/chapters/253164?view_adult=true
http://archiveofourown.org/works/729134/chapters/1394074?view_adult=true
http://archiveofourown.org/works/729134/chapters/3373481?view_adult=true


In [252]:
for item in df_all.sort_values(by = ['Kudos','Topic_novelty'])[-10:]['URL']:
    print(item)

http://archiveofourown.org/works/342490/chapters/558615
http://archiveofourown.org/works/342490/chapters/562480
http://archiveofourown.org/works/342490/chapters/557836
http://archiveofourown.org/works/342490/chapters/556681
http://archiveofourown.org/works/342490/chapters/561433
http://archiveofourown.org/works/342490/chapters/555348
http://archiveofourown.org/works/342490/chapters/560654
http://archiveofourown.org/works/173274/chapters/253164?view_adult=true
http://archiveofourown.org/works/729134/chapters/3373481?view_adult=true
http://archiveofourown.org/works/729134/chapters/1394074?view_adult=true
